In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_timestamp, col, regexp_extract, make_date, lit, when

In [0]:
spark = SparkSession.builder.appName("silver_visitors_agegroup").getOrCreate()

In [0]:
# read data
bronze_df = spark.table("workspace.growth_poc.bronze_visitors_agegroup")

key_columns = ["연령별"]
# get all non-key columns -> will be used for dynamic sql
columns_to_be_unpivoted = ["`"+c+"`" for c in bronze_df.columns if c not in key_columns]

# build the dynamic column list 
unpivot_columns = ", ".join(columns_to_be_unpivoted)

dynamic_sql = f"""
SELECT REPLACE(`연령별`, " ", "") AS age_group,
    visit_year_kor,
    amount
FROM workspace.growth_poc.bronze_visitors_agegroup v 
UNPIVOT (
    amount for visit_year_kor in ({unpivot_columns})
)
"""

# Execute the dynamic SQL
unpivoted_df = spark.sql(dynamic_sql).filter((col("visit_year_kor") != "계") & \
                                            (~col("age_group").isin(["승무원", "전체"]))) # exclude total value


In [0]:
extract_year_df = unpivoted_df.select(
    "*",
    regexp_extract(col("visit_year_kor"), r"^(\d{4})", 1).alias("visit_year"),
    make_date(regexp_extract(col("visit_year_kor"), r"^(\d{4})", 1), lit(1), lit(1)).alias("visit_date"),
    regexp_extract(col("age_group"), r"^(\d{2}|\d{1})", 1).alias("min_age"),
    when(col("age_group").endswith("~"), lit(80)) \
        .otherwise(regexp_extract(col("age_group"), r"(\d{2})$", 1)).alias("max_age")
 )

In [0]:
clean_agegroup_df = extract_year_df.withColumn("mid_age", (col("min_age")+col("max_age"))/2)

In [0]:
# add timestamp
final_df = clean_agegroup_df.withColumn("TimeStamp", current_timestamp())

final_df.write \
        .format("delta") \
        .mode("overwrite")\
        .option("mergeSchema", True)\
        .saveAsTable("workspace.growth_poc.silver_visitors_agegroup")